In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
import nltk, os, re, string, collections
import matplotlib
from  imageio import imread
from nltk.util import ngrams
import itertools
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from collections import Counter
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import gc
from mlxtend.classifier import StackingClassifier
from bayes_opt import BayesianOptimization #pip install bayesian_optimization
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from simpletransformers.classification import ClassificationModel
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion

In [2]:
train = pd.read_csv('original_data/train.csv')
test  = pd.read_csv('original_data/test.csv')

# 1) LIMPIEZA DE TEXTO

## 1) a) Funciones

In [3]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [4]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

In [5]:
# FUNCIONES AUXILIARES

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def prettify(word):
    word = re.sub(r'[^A-Za-z]', " ",  word)
    word = re.sub(r'\b\w\b',' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word

def cleanStopwords(tweet):
    tweet = tweet.split(' ')
    cleanTweet=""
    for word in tweet:
        if word not in stopwordsList:
            cleanTweet = cleanTweet + " " + word
    return cleanTweet

def clean_text_non_strict(text): 
    tw = " "
    words = text.split(' ')
    for word in words:
        word = ''.join(filter(lambda x: x in set(string.printable), word))
        word = word.replace("\n"," ")
        word = word.replace('û',"")
        word = word.replace('Û',"")
        #word = word.replace('_','')
        #word = word.replace("\"",'')
        #word = word.strip('.')
        #word = word.strip(',')
        #word = word.strip(':')
        tw += word + " "
    return tw

In [6]:
nlp = spacy.load('en', disable=['parser', 'ner'])
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

In [7]:
#FUNCIONES AUXILIARES: Limpieza de tweet

# Función que saca los links

def quitar_link_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if (("http" not in w) and ("https" not in w)):
            res.append(w)
    return (" ").join(res)

# Función que saca las menciones

def quitar_mencion_twitter(tweet):
    res = []
    for w in tweet.split(" "):
        if("@" not in w):
            res.append(w)
    return (" ").join(res)

def quitar_simbolo(tweet, simbolo):
    text = ""
    for c in tweet:
        if(c!=simbolo):
            text+=c
    return text

def agregar_espacio_ente_comas(tweet):
    
    tweer_sin_coma = (" ").join(tweet.split(","))

    return (" ").join(tweer_sin_coma.split("."))

def capitalize_each_word(string):
    
    text = ""
    for w in string.split(" "):
        if(len(w)>3):
            text += w.capitalize()+" "
    return text

def limpiar_tweet_location(tweet):
    res = tweet
    func = [quitar_mencion_twitter, quitar_link_twitter, agregar_espacio_ente_comas, lambda x: quitar_simbolo(x, "#"), capitalize_each_word]
    for f in func:
        res = f(res)
    return res

def to_string(a_list, delimeter):
    res = ""
    
    for e in sorted(filter(None, a_list)):
        res += str(e)+delimeter
    
    if(len(res)==0): return None
    return res[: -len(delimeter)]

## Sólo Texto [mejor hasta ahora]

In [10]:
#test.text  = test["text"].apply(lambda x: quitar_link_twitter(x))
#test.text  = test["text"].apply(lambda x: quitar_mencion_twitter(x))

test.text  = test["text"].apply(lambda x: clean_text_non_strict(x))
test.text  = test["text"].apply(lambda x: clean_text_strict(x))

In [11]:
test.text  = test.text.apply(prettify)

In [12]:
# a veces baja score
#test.text  = test.text.apply(lemmatize_text_v2)

In [13]:
test.text  = test.text.apply(cleanStopwords)

In [14]:
test.text

0                                         terrible crash 
1                           heard earthquake cities safe 
2             forest fire spot pond geese fleeing street 
3                  apocalypse lighting spokane wildfires 
4                    typhoon soudelor kills china taiwan 
                              ...                        
3258      earthquake safety los angeles safety fastene...
3259      storm ri hurricane city hardest hit yard bom...
3260                 green derailment chicago utbxlcbiuy 
3261             meg issues hazardous outlook hwo rbqjhn 
3262      cityofcalgary activated municipal emergency ...
Name: text, Length: 3263, dtype: object

# Modelo

In [15]:
bert_args = {
    'gradient_accumulation_steps': 2,
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 2,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False
}

In [16]:
model_bert = ClassificationModel('bert', 'models/bert_model/best_model', args=bert_args,use_cuda=False, num_labels=2)

# PREDICT ON TEST AND SAVE

In [18]:
predictions, raw_outputs = model_bert.predict(test['text'])

In [19]:
test['target'] = predictions
test.tail()

,id,keyword,location,text,target
3258,10861,NaN,NaN,earthquake safety los angeles safety fastene...,1
3259,10865,NaN,NaN,storm ri hurricane city hardest hit yard bom...,1
3260,10868,NaN,NaN,green derailment chicago utbxlcbiuy,1
3261,10874,NaN,NaN,meg issues hazardous outlook hwo rbqjhn,1
3262,10875,NaN,NaN,cityofcalgary activated municipal emergency ...,1


In [20]:
prediction_csv = test[['id','target']]
prediction_csv = prediction_csv.set_index('id')

In [21]:
prediction_csv

,target
id,
0,0
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [22]:
prediction_csv.to_csv('predictions/bert_not_lemmatized_20200808_1909.csv')